In [ ]:
# Import statements


In [2]:
import json

import pandas as pd
import numpy as np

from scipy.io import mmread

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import adjusted_rand_score
from sklearn.decomposition import PCA
from sklearn.metrics import adjusted_rand_score

from yellowbrick.cluster import SilhouetteVisualizer

import umap
import umap.plot as uplot

from kneed import KneeLocator
import random

random.seed(12)

sns.set_theme()

/opt/anaconda3/envs/venvum/lib/python3.10/site-packages/numba/np/ufunc/dufunc.py:343: NumbaWarning: Compilation requested for previously compiled argument types ((uint32,)). This has no effect and perhaps indicates a bug in the calling code (compiling a ufunc more than once for the same signature
  warnings.warn(msg, errors.NumbaWarning)
/opt/anaconda3/envs/venvum/lib/python3.10/site-packages/numba/np/ufunc/dufunc.py:343: NumbaWarning: Compilation requested for previously compiled argument types ((uint32,)). This has no effect and perhaps indicates a bug in the calling code (compiling a ufunc more than once for the same signature
  warnings.warn(msg, errors.NumbaWarning)
/opt/anaconda3/envs/venvum/lib/python3.10/site-packages/numba/np/ufunc/dufunc.py:343: NumbaWarning: Compilation requested for previously compiled argument types ((uint32,)). This has no effect and perhaps indicates a bug in the calling code (compiling a ufunc more than once for the same signature
  warnings.warn(msg, e

Data loading

In [ ]:
# Single-cell RNA sequencing (scRNA-seq) is a technique used to study the gene expression patterns of individual cells. 


In [11]:
# Load the dataset
cell_names_path = 'barcodes.tsv'
gene_names_path = 'genes.tsv'
data_path = 'matrix.mtx'

cell_names = pd.read_csv(cell_names_path, sep='\t', header=None)[0]
gene_names = pd.read_csv(gene_names_path, sep='\t', header=None)[1]

In [12]:
# Load cell annotations
cell_annotations_path = 'pmbc_3k_cell_types.json'
fle = open(cell_annotations_path, 'r')
cell_type_dict = json.load(fle)
fle.close()

In [13]:
cell_type_dict

{'AAACATACAACCAC-1': 'Memory CD4 T',
 'AAACATTGAGCTAC-1': 'B-cells',
 'AAACATTGATCAGC-1': 'Memory CD4 T',
 'AAACCGTGCTTCCG-1': 'CD14+ Mono',
 'AAACCGTGTATGCG-1': 'NK cells',
 'AAACGCACTGGTAC-1': 'Memory CD4 T',
 'AAACGCTGACCAGT-1': 'CD8 T',
 'AAACGCTGGTTCTT-1': 'CD8 T',
 'AAACGCTGTAGCCA-1': 'Naive CD4 T',
 'AAACGCTGTTTCTG-1': 'FCGR3A+ Mono',
 'AAACTTGAAAAACG-1': 'B-cells',
 'AAACTTGATCCAGA-1': 'Naive CD4 T',
 'AAAGAGACGAGATA-1': 'Naive CD4 T',
 'AAAGAGACGCGAGA-1': 'CD14+ Mono',
 'AAAGAGACGGACTT-1': 'Naive CD4 T',
 'AAAGAGACGGCATT-1': 'Naive CD4 T',
 'AAAGCAGATATCGG-1': 'CD14+ Mono',
 'AAAGCCTGTATGCG-1': 'Memory CD4 T',
 'AAAGGCCTGTCTAG-1': 'B-cells',
 'AAAGTTTGATCACG-1': 'B-cells',
 'AAAGTTTGGGGTGA-1': 'B-cells',
 'AAAGTTTGTAGAGA-1': 'Naive CD4 T',
 'AAAGTTTGTAGCGT-1': 'CD14+ Mono',
 'AAATCAACAATGCC-1': 'B-cells',
 'AAATCAACACCAGT-1': 'Naive CD4 T',
 'AAATCAACCAGGAG-1': 'Naive CD4 T',
 'AAATCAACCCTATT-1': 'FCGR3A+ Mono',
 'AAATCAACGGAAGC-1': 'Naive CD4 T',
 'AAATCAACTCGCAA-1': 'Memory 

In [14]:
# Show cell names
cell_names.head()

0    AAACATACAACCAC-1
1    AAACATTGAGCTAC-1
2    AAACATTGATCAGC-1
3    AAACCGTGCTTCCG-1
4    AAACCGTGTATGCG-1
Name: 0, dtype: object

In [15]:
# Show gene names
gene_names.head()

0      MIR1302-10
1         FAM138A
2           OR4F5
3    RP11-34P13.7
4    RP11-34P13.8
Name: 1, dtype: object

In [17]:
# Load expression matrix
expression_mtx = mmread(data_path)

ValueError: Line 1574857: Invalid integer value.

In [18]:
# Inspect file content
with open(data_path, 'r') as f:
    lines = f.readlines()

# Print the problematic line
print(lines[1574856])  # Line numbers in Python are zero-based


2622


In [19]:
import pandas as pd

expression_mtx = pd.read_csv(data_path)


In [20]:
expression_mtx

,%%MatrixMarket matrix coordinate real general
0,%
1,32738 2700 2286884
2,32709 1 4
3,32707 1 1
4,32706 1 10
...,...
1574851,26339 1861 8
1574852,26319 1861 2
1574853,26308 1861 1
1574854,26246 1861 1


In [21]:
# Show shapes
print('Expression shape : {}'.format(expression_mtx.shape))
print('Num cells : {}'.format(cell_names.shape[0]))
print('Num genes : {}'.format(gene_names.shape[0]))

Expression shape : (1574856, 1)
Num cells : 2700
Num genes : 32738


In [22]:
# Create dataframe from sparse matrix
expression_df = pd.DataFrame.sparse.from_spmatrix(
    expression_mtx, 
    columns=cell_names.to_numpy(), 
    index=gene_names.to_numpy()
)

AttributeError: 'DataFrame' object has no attribute 'tocsc'

In [23]:
print(type(expression_mtx))


<class 'pandas.core.frame.DataFrame'>


In [24]:
from scipy.io import mmread
expression_mtx = mmread('matrix.mtx')
print(type(expression_mtx))  # This should now show a sparse matrix type


ValueError: Line 1574857: Invalid integer value.

In [25]:
valid_lines = []
with open('matrix.mtx', 'r') as file:
    for line in file:
        try:
            parts = line.strip().split()
            if len(parts) == 3:
                int(parts[0]), int(parts[1]), float(parts[2])  # Validate types
            valid_lines.append(line)
        except ValueError:
            continue  # Skip invalid lines

with open('clean_matrix.mtx', 'w') as out_file:
    out_file.writelines(valid_lines)
